In [51]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
from tensorflow import keras
from gensim.models import KeyedVectors
from jieba.analyse import extract_tags, textrank

tf.get_logger().setLevel('ERROR')
print(tf.__version__)
tf.config.list_physical_devices('GPU')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2.10.1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [55]:
from utils.preprocessing import preprocess

raw_ds = preprocess('word2vec')

Total number of news: 30339
Maximum number of news for each group in training set: 64
Minimum number of news for each group in training set: 3
Maximum number of news for each group in test set: 26
Minimum number of news for each group in test set: 3
Maximum number of times a news appears in training set: 22
Minimum number of times a news appears in training set: 0
Maximum number of times a news appears in test set: 11
Minimum number of times a news appears in test set: 0
Loading word2vec model...


In [29]:
%%time
cn_model = KeyedVectors.load_word2vec_format(r'D:\fd\semester5\PRML\PJ\assignment2\Chinese-Word-Vectors-master\tencent-ailab-embedding-zh-d100-v0.2.0\tencent-ailab-embedding-zh-d100-v0.2.0.txt', binary=False)

CPU times: total: 15min 45s
Wall time: 15min 46s


In [49]:
title, content = 1, 3

In [47]:
mat = [[cn_model[word]*weight for word, weight in (extract_tags(raw_ds['news'][ID][0], topK=title, withWeight=True)+textrank(raw_ds['news'][ID][1], topK=content, withWeight=True))] for ID in raw_ds['news'].keys()]

KeyError: "Key '通大手' not present"

In [57]:
mat = [[cn_model[word]*weight for word, weight in textrank(raw_ds['news'][ID][0], topK=title, withWeight=True)] for ID in raw_ds['news'].keys()]

KeyError: "Key '低炭' not present"